# PR Cycle

Enter the reference you would like to add (e.g `JIRA-123`)

In [12]:
from datetime import date, datetime, timedelta
import logging
from functools import cache
from IPython.display import HTML

import pandas as pd
import ipywidgets as widgets
import plotly.express as px

from prfiesta.collectors.github import GitHubCollector
from prfiesta.analysis.view import view_pr_cycle, _enrich_pr_link

logging.basicConfig(format=logging.BASIC_FORMAT, level=logging.INFO)
logger = logging.getLogger()

In [7]:
@cache
def create_collector() -> GitHubCollector:
    return GitHubCollector()

def collect_dummy_reference(reference: str):
    logger.warning("USING DUMMY DATA")

    df = pd.DataFrame(
        data={
            "number": range(5), 
            "title": [
                "Add feature X",
                "Fix bug in Y",
                "Refactor Z component",
                "Improve performance in A",
                "Update documentation for B",
            ],
            "repository_name": ["repository"] * 5,
            "html_url": ["url" + str(i) for i in range(1, 6)],
            "created_at": [
                datetime(2023, 1, 1) + timedelta(days=i*2) for i in range(5)
            ],  # Spread across a month
            "pull_request.merged_at": [
                datetime(2023, 1, 3) + timedelta(days=i*3) for i in range(5)
            ],
        }
    )
    df["cycle_time"] = df["pull_request.merged_at"] - df["created_at"]
    df["cycle_time_mins"] = df["cycle_time"].dt.total_seconds() / 60
    df["cycle_time_hours"] = df["cycle_time_mins"] / 60
    df['reference'] = reference
    
    return df

# TODO: Add Cache to this
@cache
def collect_reference(reference: str, dummy: bool) -> pd.DataFrame:
    logger.info('collecting %s', reference)
    
    if dummy:
        df = collect_dummy_reference(reference)
    else:
        collector = create_collector()
        df = collector.collect(reference=reference)
        df = view_pr_cycle(df, as_frame=True)
        #df = _enrich_pr_link(df)
        
    return df

In [8]:
## STATE

current_references: set = set()
current_results: list[pd.DataFrame] = []
all_results: pd.DataFrame = pd.DataFrame()

def clear_state():
    current_references.clear()
    current_results.clear()
    all_results = pd.DataFrame()

In [9]:
## OUTPUTS

control_output = widgets.Output()
result_frame_output = widgets.Output()
statistics_output = widgets.Output()
gantt_chart_output = widgets.Output()

def clear_ui_state():
    control_output.clear_output()
    result_frame_output.clear_output()
    statistics_output.clear_output()
    gantt_chart_output.clear_output()

In [10]:
## UI ELEMENTS

text_input = widgets.Text(placeholder='e.g JIRA-123', description='Reference:', value='PRFIESTA-123')

add_button = widgets.Button(description='Add', button_style='success')
clear_button = widgets.Button(description='Clear', button_style='danger')

style = """
    <style>
    .stats-header {
        color: white;
        background-color: teal;
        border-radius: 2px;
        text-align: center;
    }

    .stats-value {
        font-size: 200%;
        display: block;
        text-align: center;
    }
    </style>
    """

In [14]:
def create_statistics_block(header: str, data: str, stats_header_css_class='stats-header', stats_value_css_class='stats-value'):
    return widgets.HTML(f'''
        <h2 class="{stats_header_css_class}">{header}</h2>
        <span class="{stats_value_css_class}">{data}</span>
    ''')

def add_reference(b):
    if not text_input.value:
        return
    
    value = str.strip(text_input.value)
    current_references.add(value)
        
    df = collect_reference(value, dummy=False)
    current_results.append(df)
    all_results = pd.concat(current_results)
    
    clear_ui_state()
        
    with control_output:
        display(widgets.TagsInput(value=list(current_references), read_only=True))
        
    with result_frame_output:
        temp = _enrich_pr_link(all_results.set_index(['repository_name', 'number']))
        display(HTML(temp.to_html(escape=False, index=False)))
        
    with statistics_output:
        
#         display(df)
#         display(df.dtypes)
        
        earliest_pr_create_time = all_results['created_at'].min().date()
        latest_merged_at = all_results['pull_request.merged_at'].max().date()
        average_cycle_time_hours = all_results['cycle_time_hours'].mean() 

        earliest_pr_create_view = create_statistics_block("Earliest PR Create Time", earliest_pr_create_time)
        latest_merged_at_view = create_statistics_block("Latest Merged At", latest_merged_at)
        average_cycle_time_hours_views = create_statistics_block("Average Cycle Time (Hours)", average_cycle_time_hours)

        component = widgets.TwoByTwoLayout(
            top_left=earliest_pr_create_view,
            bottom_left=latest_merged_at_view,
            bottom_right=average_cycle_time_hours_views
        )
        
        display(component)
        
    with gantt_chart_output:
        fig = px.timeline(df, x_start="created_at", x_end="pull_request.merged_at", y="title", )
        fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
        fig.show()
        
def clear_references(b):
    if not current_references:
        return
    
    clear_ui_state()
    clear_state()
        
add_button.on_click(add_reference)
clear_button.on_click(clear_references)


display(
    widgets.HTML(style),
    widgets.HBox([text_input, add_button, clear_button]),
    control_output,
    widgets.AppLayout(
        header=gantt_chart_output,
        left_sidebar=None,
        center=result_frame_output,
        right_sidebar=statistics_output,
        footer=None
    )
)

HTML(value='\n    <style>\n    .stats-header {\n        color: white;\n        background-color: teal;\n      …

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "TagsInput(value=['PRFIESTA-123'])", 'a…

AppLayout(children=(Output(layout=Layout(grid_area='header'), outputs=({'output_type': 'display_data', 'data':…